In [17]:
# !pip install pandas_datareader
import boto3
import base64
import sagemaker
from botocore.exceptions import ClientError
from IPython.display import display
import pandas_datareader
import pandas as pd
import numpy as np
from time import sleep
import os
import requests
import csv

In [18]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
prefix = 'stock-data-raw'
tickers_url = 'http://ftp.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt'
# tickers = ['AAPL', 'NVDA'] ## to be changed to the entire list at http://ftp.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt

Couldn't call 'get_role' to get Role ARN from role name AmazonSageMaker-ExecutionRole-20210310T221946 to get Role path.
Assuming role was created in SageMaker AWS console, as the name contains `AmazonSageMaker-ExecutionRole`. Defaulting to Role ARN with service-role in path. If this Role ARN is incorrect, please add IAM read permissions to your role or supply the Role Arn directly.


In [62]:
## to do for massive data base; keeping small for now. 
def get_all_tickers(tickers_url):
    text = requests.get(tickers_url).text
    with open('temp.csv', 'w') as f:
        f.write(text)
    df = pd.read_csv('temp.csv', delimiter='|')
    tickers = df.Symbol.tolist()
    return tickers
tickers = get_all_tickers(tickers_url)[:150]
display(tickers)

['AACG',
 'AACQ',
 'AACQU',
 'AACQW',
 'AAL',
 'AAME',
 'AAOI',
 'AAON',
 'AAPL',
 'AAWW',
 'AAXJ',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABGI',
 'ABIO',
 'ABMD',
 'ABNB',
 'ABST',
 'ABTX',
 'ABUS',
 'ACAC',
 'ACACU',
 'ACACW',
 'ACAD',
 'ACAHU',
 'ACBI',
 'ACCD',
 'ACER',
 'ACET',
 'ACEV',
 'ACEVU',
 'ACEVW',
 'ACGL',
 'ACGLO',
 'ACGLP',
 'ACHC',
 'ACHL',
 'ACHV',
 'ACIU',
 'ACIW',
 'ACKIT',
 'ACKIU',
 'ACKIW',
 'ACLS',
 'ACMR',
 'ACNB',
 'ACOR',
 'ACQRU',
 'ACRS',
 'ACRX',
 'ACST',
 'ACTC',
 'ACTCU',
 'ACTCW',
 'ACTDU',
 'ACTG',
 'ACVA',
 'ACWI',
 'ACWX',
 'ADAG',
 'ADAP',
 'ADBE',
 'ADER',
 'ADERU',
 'ADERW',
 'ADES',
 'ADI',
 'ADIL',
 'ADILW',
 'ADMA',
 'ADMP',
 'ADMS',
 'ADN',
 'ADNWW',
 'ADOC',
 'ADOCR',
 'ADOCW',
 'ADP',
 'ADPT',
 'ADRE',
 'ADSK',
 'ADTN',
 'ADTX',
 'ADUS',
 'ADV',
 'ADVM',
 'ADVWW',
 'ADXN',
 'ADXS',
 'AEAC',
 'AEACU',
 'AEACW',
 'AEGN',
 'AEHL',
 'AEHR',
 'AEI',
 'AEIS',
 'AEMD',
 'AEP',
 'AEPPL',
 'AEPPZ',
 'AERI',
 'AESE',
 'AEY',
 'AEYE',
 'AEZS',
 'AFAQU',

In [54]:
def get_secret():
    secret_name = "alpha_vantage"
    region_name = "us-east-2"
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except Exception as e:
        display(e)
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            secret = base64.b64decode(get_secret_value_response['SecretBinary'])
    return secret   

def filter_complete_tickers(tickers, bucket, prefix):
    file_names = sess.list_s3_files(bucket, prefix)
    new = []
    for ticker in tickers:
        ticker_file_name = os.path.join(prefix, f'{ticker}_daily_raw.parquet')
        if ticker_file_name not in file_names:
            new.append(ticker)
    return new
                        
def read_data_to_s3(tickers, ALPHA_API_KEY, bucket, prefix):
    tickers = filter_complete_tickers(tickers, bucket, prefix)
    i = 0
    for ticker in tickers:      
        try: ## TODO: switch from error handling to simple check if exists; no pull of data
            pandas_datareader.av.time_series.AVTimeSeriesReader(symbols=ticker, api_key=ALPHA_API_KEY, function='TIME_SERIES_DAILY').read().to_parquet('temp.parquet')
            boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, f'{ticker}_daily_raw.parquet')).upload_file('temp.parquet')
            ## logic for api waiting
            i+=1
            if i%5==0: ## for 5 requests per minute limit on api
                sleep(61)
            if i%500==0:
                sleep(24*60*60+1) ## 1 day for 500 requests per day limit on api
#             pd.read_pickle(f'./{ticker}_daily_raw.pkl')
        except Exception as e:
            display(e)
            break
        display(f'{ticker} done')       

In [55]:
# ALPHA_API_KEY = get_secret()

In [59]:
read_data_to_s3(tickers, ALPHA_API_KEY, bucket, prefix)

'ACACU done'

'ACACW done'

'ACAD done'

'ACAHU done'

'ACBI done'

...

'ADUS done'

'ADV done'

'ADVM done'

'ADVWW done'

'ADXN done'

In [61]:
## test whether upload worked
display(len(sess.list_s3_files(bucket, prefix)))
# sess.download_data('./test.parquet',bucket, prefix)

# df = pd.read_parquet('test.parquet')

# diplay(df)

91